#### <center> Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

### <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам <br>Неделя 1. Подготовка данных к анализу и построению моделей <center>

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы `Scipy.sparse`), который хорошо подходит для данной задачи. 

**План 1 недели:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

In [1]:
from __future__ import division, print_function
# отключим предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
def write_answer_to_file(answer, textfile):
    with open(textfile, 'w') as fout:
        fout.write(str(answer))

In [3]:
#путь к данным
PATH_TO_DATA = 'capstone_user_identification'

**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами *path_to_csv_files* и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

In [4]:
#функция для создания отсортированного частотного словаря (меньшему индексу - большая частота)
def create_freq_dict(path_to_csv_files):
    
    freq_dict = {}
    
    for path_to_file in glob(path_to_csv_files + '/*.csv'):
        with open(path_to_file) as file: 
            lines = file.readlines()[1:]
        
            #заполнение словаря key: site, value: site_id, count
            for i, line in enumerate(lines):
                site = line.strip().split(',')[1]
                count = 1
                if site in freq_dict:
                    count = freq_dict[site] + 1
                freq_dict.update({site: count})
    
    #сортировка словаря
    sorted_tuples = sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)
    sorted_dict = {(k): (i+1, v) for i, (k,v) in enumerate(sorted_tuples)}
    
    return sorted_dict

In [5]:
def prepare_train_set(path_to_csv_files, session_length=10):
    
    window_size = 10 #ширина окна
    site_freq_dict = {} # частотный словарь
    data = [] #массив с индексами сайтов по сессиям
    sites_seq = [] #список сайтов для каждого пользователя    
    user_ids = [] #массив id пользователей
    start_session_index = 0 #индекс начала следующей сессии в зависимости от ширины окна 
    
    site_freq_dict = create_freq_dict(path_to_csv_files)
    
    for user_id, path_to_file in enumerate(glob(path_to_csv_files + '/*.csv')):
        user_id += 1
        with open(path_to_file) as file: 
            lines = file.readlines()[1:]
        
            for i, line in enumerate(lines):
                site = line.strip().split(',')[1]
                sites_seq.append(site)
                                
        #заполнение массива id сайтов
        while start_session_index < len(lines):
            session_end = len(lines)\
                        if start_session_index + session_length > len(lines) - 1 \
                        else start_session_index + session_length
            sites_id = list(site_freq_dict[i][0] for i in sites_seq[start_session_index:session_end])
            data.append(sites_id)
            user_ids.append(user_id)
            start_session_index += window_size            
                    
        start_session_index = 0  
        sites_seq = []
    
    #создание датафрейма 
    df = pd.DataFrame(data, columns = ['site' + str(i) for i in range(1, session_length + 1)], dtype=object)
    df = df.fillna(0)
    df['user_id'] = user_ids                    
    
    return df, site_freq_dict

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [6]:
!cat $PATH_TO_DATA/3users/user0001.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,geo.mozilla.org
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,accounts.google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,apis.google.com
2013-11-15 11:41:35,plus.google.com
2013-11-15 12:40:35,vk.com
2013-11-15 12:40:37,google.com
2013-11-15 12:40:40,google.com
2013-11-15 12:41:35,google.com


In [7]:
!cat $PATH_TO_DATA/3users/user0002.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,football.kulichki.ru
2013-11-15 11:37:26,football.kulichki.ru
2013-11-15 11:40:32,oracle.com


In [8]:
!cat $PATH_TO_DATA/3users/user0003.csv

timestamp,site
2013-11-15 09:28:17,meduza.io
2013-11-15 09:33:04,google.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,google.com
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,yandex.ru
2013-11-15 11:41:35,meduza.io
2013-11-15 12:28:17,meduza.io
2013-11-15 12:33:04,google.com
2013-11-15 12:52:48,oracle.com


In [9]:
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

In [10]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,3,2,2,7,2,1,8,5,9,10,1
1,3,1,1,1,0,0,0,0,0,0,1
2,3,2,6,6,2,0,0,0,0,0,2
3,4,1,2,1,2,1,1,5,11,4,3
4,4,1,2,0,0,0,0,0,0,0,3


Частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой (первые элементы кортежей могут отличаться).

In [11]:
site_freq_3users

{'google.com': (1, 9),
 'oracle.com': (2, 8),
 'vk.com': (3, 3),
 'meduza.io': (4, 3),
 'mail.google.com': (5, 2),
 'football.kulichki.ru': (6, 2),
 'geo.mozilla.org': (7, 1),
 'accounts.google.com': (8, 1),
 'apis.google.com': (9, 1),
 'plus.google.com': (10, 1),
 'yandex.ru': (11, 1)}

Примените полученную функцию к данным по 10 пользователям.

**<font color='red'> Вопрос 1. </font> Сколько уникальных сессий из 10 сайтов в выборке с 10 пользователями?**

In [12]:
%%time
train_data_10users, site_freq_10users = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 
                                                     session_length=10)

CPU times: user 175 ms, sys: 2.8 ms, total: 178 ms
Wall time: 177 ms


In [13]:
train_data_10users.shape[0]

14061

In [14]:
write_answer_to_file(train_data_10users.shape[0], '1-1.txt')

**<font color='red'> Вопрос 2. </font> Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [15]:
len(site_freq_10users)

4913

In [16]:
write_answer_to_file(len(site_freq_10users), '1-2.txt')

Примените полученную функцию к данным по 150 пользователям.

**<font color='red'> Вопрос 3. </font> Сколько уникальных сессий из 10 сайтов в выборке с 150 пользователями?**

In [17]:
%%time
train_data_150users, site_freq_150users = prepare_train_set(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)

CPU times: user 1.71 s, sys: 54.5 ms, total: 1.77 s
Wall time: 1.76 s


In [18]:
train_data_150users.shape[0]

137019

In [19]:
write_answer_to_file(train_data_150users.shape[0], '1-3.txt')

**<font color='red'> Вопрос 4. </font> Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [20]:
len(site_freq_150users)

27797

In [21]:
write_answer_to_file(len(site_freq_150users), '1-4.txt')

**<font color='red'> Вопрос 5. </font> Какой из этих сайтов НЕ входит в топ-10 самых популярных сайтов среди посещенных 150 пользователями?**
- www.google.fr
- www.youtube.com
- safebrowsing-cache.google.com
- www.linkedin.com

In [22]:
list(site_freq_150users.items())[:10]

[('www.google.fr', (1, 64785)),
 ('www.google.com', (2, 51320)),
 ('www.facebook.com', (3, 39002)),
 ('apis.google.com', (4, 29983)),
 ('s.youtube.com', (5, 29102)),
 ('clients1.google.com', (6, 25087)),
 ('mail.google.com', (7, 19072)),
 ('plus.google.com', (8, 18467)),
 ('safebrowsing-cache.google.com', (9, 17960)),
 ('www.youtube.com', (10, 16319))]

In [23]:
top10_popular = list(site_freq_150users.keys())[:10]
top10_popular

['www.google.fr',
 'www.google.com',
 'www.facebook.com',
 'apis.google.com',
 's.youtube.com',
 'clients1.google.com',
 'mail.google.com',
 'plus.google.com',
 'safebrowsing-cache.google.com',
 'www.youtube.com']

In [24]:
write_answer_to_file(' '.join(top10_popular), '1-5.txt')

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [25]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. 

Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить. 

In [26]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [27]:
X_toy

array([[ 3,  2,  2,  7,  2,  1,  8,  5,  9, 10],
       [ 3,  1,  1,  1,  0,  0,  0,  0,  0,  0],
       [ 3,  2,  6,  6,  2,  0,  0,  0,  0,  0],
       [ 4,  1,  2,  1,  2,  1,  1,  5, 11,  4],
       [ 4,  1,  2,  0,  0,  0,  0,  0,  0,  0]])

In [28]:
y_toy

array([1, 1, 2, 3, 3])

In [29]:
#создание функции, которая подготавливает данные data, indices, indptr для csr_matrix 
def data_for_csr_matrix(data):
    indptr = [0]
    indices = []
    sparse_data = []
    vocabulary = {}
    for d in data:
        for term in d:
            if term!=0:
                index = vocabulary.setdefault(term, len(vocabulary))
                indices.append(index) 
                sparse_data.append(1)
        indptr.append(len(indices))
    return sparse_data, indices, indptr

In [30]:
X_sparse_toy = csr_matrix(data_for_csr_matrix(X_toy), dtype=int)  

**Размерность разреженной матрицы должна получиться равной 11, поскольку в игрушечном примере 3 пользователя посетили 11 уникальных сайтов.**

In [31]:
X_sparse_toy.todense()

matrix([[1, 3, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 2, 0, 4, 0, 1, 0, 0, 0, 2, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]])

In [32]:
X_sparse_toy.shape

(5, 11)

In [33]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [34]:
X_sparse_10users = csr_matrix(data_for_csr_matrix(X_10users), dtype=int)
X_sparse_150users = csr_matrix(data_for_csr_matrix(X_150users), dtype=int)

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [35]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

In [36]:
with open(os.path.join(PATH_TO_DATA, 'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)

**Проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [37]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [38]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)